In [1]:
import numpy as np
from scipy.sparse.linalg import svds
from sklearn import datasets

In [20]:
def e(i, d):
    ei = np.zeros(d)
    ei[i] = 1
    return ei



def KWSA(F, w, c, d):
    """ 
    Kiefer-Wolfowitz stochastic approximation
    for gradient estimation 
    
    INPUT:
    - F: objective function
    - w: current weight
    - d: dimension
    - c: costant
    
    """
    np.ones(d)
    g = np.zeros(d)
    for i in range(d):
        g[i] = (F(w + c * e(i, d)) - F(w))/c
    return g



def detZFW(F, L, d, w0, r=1, T=100, eps=1e-5):
    """
    INPUT
    - F: loss function
    - L: Lip constant
    - d: dimension
    - w0: starting point
    - r: radius of the ball
    - T: max Iteration
    - eps: tolerance
    """

    gamma = lambda t: 2/(t+2)
    c = lambda t: L*gamma(t)/d
    w = w0
    partial = 0
    for t in range(1, T):
        # comupute the gradient approx
        gt = KWSA(F, w, c(t), d)
        # comupute the linear problem solution ON L1 BALL
        ei = e(np.argmax(np.abs(gt)), d)
        v = np.sign(gt) * ei
        # compute step 
        w_pred = w
        w = (1 - gamma(t)) * w + gamma(t) * v
        loss_eval = np.abs(F(w) - F(w_pred))
        print(f"Loss evaluation at time {t}:\t{loss_eval:.2f}\n")
        print(w)
        partial += w
        # check stopping condition
        if loss_eval < eps:
            break
    return F(w_pred), F(w), w, partial/T, t

## Stochastic lasso regression

In [3]:
# load data
X, y = datasets.load_svmlight_file("../Data/covtype.libsvm.binary.scale.bz2")

In [4]:
# space dimension
d = X.shape[1]
print(f"Space Dimensions\nd: {d}")
print(f"n: {y.shape[0]}")

Space Dimensions
d: 54
n: 581012


In [5]:
# define the objective function
F = lambda w: 0.5 * sum(np.power(y - X @ w, 2))

In [18]:
# initialize prarameters for the algorithm

# stating point - TODO: randomize
w0 = np.random.rand(d)
w0 = w0/sum(w0)
#print(w0)
#print(F(w0))

# Lipschitz constant computation
L = 2/X.shape[0] * np.linalg.norm( X.T @ X )
#print(L/52)
L = 100 # which value consider???

In [114]:
X.T @ X 

<54x54 sparse matrix of type '<class 'numpy.float64'>'
	with 1174 stored elements in Compressed Sparse Column format>

In [7]:
fpred, f, w, mean, t = detZFW(F, L, d, w0)

print('F(w_pred) =', fpred, '\n',
      'F(w) =', f, '\n',
      'w =', w, '\n',
      'mean w =', mean, '\n',
      't =', t)

Loss evaluation at time 0:	1029722.59845063

Loss evaluation at time 1:	37980.44497389509

Loss evaluation at time 2:	10259.115671192994

Loss evaluation at time 3:	4189.21476412029

Loss evaluation at time 4:	2112.943489674246

Loss evaluation at time 5:	1212.884637331823

Loss evaluation at time 6:	760.0798483171966

Loss evaluation at time 7:	507.58611771278083

Loss evaluation at time 8:	355.7226021774113

Loss evaluation at time 9:	258.92062291968614

Loss evaluation at time 10:	194.30832990887575

Loss evaluation at time 11:	149.53669072431512

Loss evaluation at time 12:	117.53505098144524

Loss evaluation at time 13:	94.05461173178628

Loss evaluation at time 14:	76.43676308705471

Loss evaluation at time 15:	62.95962927560322

Loss evaluation at time 16:	52.4744363501668

Loss evaluation at time 17:	44.19469472696073

Loss evaluation at time 18:	37.56958400900476

Loss evaluation at time 19:	32.20549619314261

Loss evaluation at time 20:	27.816061433171853

Loss evaluation at 

KeyboardInterrupt: 